# Contexte du projet

Votre entreprise vous demande de disposer, en base de données, des titres des articles du site 20 Minutes.

![voir l'image jointe](Images/vue_site_20minutes.png)

Chaque jour, un script ira collecter ces titres, ainsi que leur catégorie, et les placera en base de données, avec un horodatage.

On doit pouvoir ainsi, par exemple, pouvoir récupérer tous les titres des articles concernant "Roland Garros", au mois de mai 2022.

## 1. Rechercher l'organisation HTML du site internet concerné, et notamment les noms et type de balise permettant de repérer les titres et catégories de articles.

Balise contenant un article : `<article>`

Dans l'article :

- `<h2 class = "teaser-title">` : Titre

- `<div class = "teaser-headline">` : Catégorie du titre

- `<span class = "counterbar-item datetime">` : Heure de l'article

---

## 2. Ecrire un script python qui récupère ces informations par scrapping, et les affiche dans la console.

In [ ]:
import requests
from bs4 import BeautifulSoup

# Récupération de la page HTML
url = "https://www.20minutes.fr/"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

articles = soup.find_all('article')

for article in articles:

    try :
        heure = article.find('span', class_ = "counterbar-item datetime").text

    except AttributeError :
        continue

    if heure.find(':') != -1:

        print(f"Titre : {article.find('h2', class_ ='teaser-title').text}")
        print(f"Catégorie : {article.find('div', class_ ='teaser-headline').text}")
        print(f"Heure : {heure}\n\n")

---

## 3. Créer une base de données avec une table articles_20minutes pour stocker date, catégorie et titre des articles.

### Définition des fonctions pour se connecter à la base de donnée et insérer des données

In [ ]:
import sqlite3 as sq
import mysql.connector as cnt

# Fonction de connexion à la base de données
def connect_db(host, user, pswd, database = None):
    db = cnt.connect(
        host = host,
        user = user,
        passwd = pswd,
        database = database
    )
    return(db)

# Fonction d'execution de requêtes SQL
def exe(db, request, value = None, verbose = False, name = False):
    cursor = db.cursor(buffered = True)
    
    if value:
        cursor.execute(request, value)

    else :
        cursor.execute(request)

    db.commit()
    
    if verbose:
        result = cursor.fetchall()
        if name:
            result.insert(0, cursor.column_names)
        return(result)

### Création de la base de données et de la table

In [ ]:
db = connect_db('192.168.20.118', 'grogu', 'grogu')

queries = ['CREATE DATABASE IF NOT EXISTS scrapping']

queries.append("""
    CREATE TABLE IF NOT EXISTS scrapping.articles_20minutes(
        Code INT NOT NULL AUTO_INCREMENT,
        Date DATETIME NOT NULL,
        Categorie TEXT NOT NULL,
        Titre TEXT NOT NULL,
        PRIMARY KEY (Code)
    )ENGINE = InnoDB;
""")

for query in queries:
    exe(db, query)

### Screenshot de la base de données

![image](Images/BDD.jpg)


---

## 4. Modifier le script python pour inscrire les données dans la base

In [ ]:
import requests, datetime
from bs4 import BeautifulSoup

# Fonction de conversion Heure:Minutes --> Datetime
def convert_time(str_time):
    liste_time = str_time.split(":")

    new_time = datetime.datetime.now()

    new_time = new_time.replace(hour = int(liste_time[0]) - 1, minute = int(liste_time[1]) - 1, second = 0)

    return(new_time)

# Scrapping

datas = []

url = "https://www.20minutes.fr/"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')


articles = soup.find_all('article')

for article in articles:

    try :
        heure = article.find('span', class_ = "counterbar-item datetime").text

    except AttributeError :
        continue

    if heure.find(':') != -1:
# Création d'une liste regroupant les données récupérées
        datas.append([convert_time(heure), article.find('div', class_ ='teaser-headline').text, article.find('h2', class_ ='teaser-title').text])


# Insertion dans la base de données
db = connect_db('192.168.20.118', 'grogu', 'grogu', database = 'scrapping')

for data in datas :

    query = f"""INSERT INTO articles_20minutes (Date, Categorie, Titre) VALUES ("{data[0]}", "{data[1]}", "{data[2]}");"""

    exe(db, query)

![image](Images/BDD_1.jpg)